In [21]:
from IPython.display import display, HTML

import pandas as pd
import numpy as np

from datetime import date

from category_encoders import TargetEncoder
from sklearn.preprocessing import StandardScaler, PowerTransformer, OneHotEncoder
from sklearn.model_selection import train_test_split


from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, root_mean_squared_error
import pickle as pkl

In [22]:
car_prices = pd.read_csv("CarPrices.csv")

car_prices.head(20)

,DateCrawled,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen,Price
0,3/24/2016 11:52,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,3/24/2016 0:00,0,70435,4/7/2016 3:16,480
1,3/24/2016 10:58,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,3/24/2016 0:00,0,66954,4/7/2016 1:46,18300
2,3/14/2016 12:52,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,3/14/2016 0:00,0,90480,4/5/2016 12:47,9800
3,3/17/2016 16:54,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,3/17/2016 0:00,0,91074,3/17/2016 17:40,1500
4,3/31/2016 17:25,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,3/31/2016 0:00,0,60437,4/6/2016 10:17,3600
5,4/4/2016 17:36,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes,4/4/2016 0:00,0,33775,4/6/2016 19:17,650
6,4/1/2016 20:48,convertible,2004,manual,109,2_reihe,150000,8,petrol,peugeot,no,4/1/2016 0:00,0,67112,4/5/2016 18:18,2200
7,3/21/2016 18:54,sedan,1980,manual,50,other,40000,7,petrol,volkswagen,no,3/21/2016 0:00,0,19348,3/25/2016 16:47,0
8,4/4/2016 23:42,bus,2014,manual,125,c_max,30000,8,petrol,ford,NaN,4/4/2016 0:00,0,94505,4/4/2016 23:42,14500
9,3/17/2016 10:53,small,1998,manual,101,golf,150000,0,NaN,volkswagen,NaN,3/17/2016 0:00,0,27472,3/31/2016 17:17,999


In [23]:
display(HTML("<h1>Data Columns</h1>"))
display(f"Columns: {car_prices.columns.to_list()}")
display(HTML("<br>"))

display(HTML("<h1>Data Information</h1>"))
car_prices.info()
display(HTML("<br>"))

display(HTML("<h1>Data Shape</h1>"))
display(car_prices.shape)

display(HTML("<h1>Data Describe</h1>"))
display(car_prices.describe())
display(HTML("<br>"))

"Columns: ['DateCrawled', 'VehicleType', 'RegistrationYear', 'Gearbox', 'Power', 'Model', 'Kilometer', 'RegistrationMonth', 'FuelType', 'Brand', 'Repaired', 'DateCreated', 'NumberOfPictures', 'PostalCode', 'LastSeen', 'Price']"

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   VehicleType        316879 non-null  object
 2   RegistrationYear   354369 non-null  int64 
 3   Gearbox            334536 non-null  object
 4   Power              354369 non-null  int64 
 5   Model              334664 non-null  object
 6   Kilometer          354369 non-null  int64 
 7   RegistrationMonth  354369 non-null  int64 
 8   FuelType           321474 non-null  object
 9   Brand              354369 non-null  object
 10  Repaired           283215 non-null  object
 11  DateCreated        354369 non-null  object
 12  NumberOfPictures   354369 non-null  int64 
 13  PostalCode         354369 non-null  int64 
 14  LastSeen           354369 non-null  object
 15  Price              354369 non-null  int64 
dtypes: int64(7), object(

(354369, 16)

,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode,Price
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000,354369.000000
mean,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087,4416.656776
std,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248,4514.158514
min,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000,0.000000
25%,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000,1050.000000
50%,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000,2700.000000
75%,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000,6400.000000
max,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000,20000.000000


In [24]:
display(HTML("<h1>Data Display after removing NaN Values</h1>"))
useless_cols = ['DateCrawled', 'DateCreated', 'LastSeen', 'NumberOfPictures']
car_prices = car_prices.drop(columns=useless_cols, errors='ignore')
display(car_prices.head(20))
display(HTML("<br>"))

display(HTML("<h1>Data NaN Values</h1>"))
display(car_prices.isna().sum())

,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,PostalCode,Price
0,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,70435,480
1,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,66954,18300
2,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,90480,9800
3,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,91074,1500
4,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,60437,3600
5,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes,33775,650
6,convertible,2004,manual,109,2_reihe,150000,8,petrol,peugeot,no,67112,2200
7,sedan,1980,manual,50,other,40000,7,petrol,volkswagen,no,19348,0
8,bus,2014,manual,125,c_max,30000,8,petrol,ford,NaN,94505,14500
9,small,1998,manual,101,golf,150000,0,NaN,volkswagen,NaN,27472,999


VehicleType          37490
RegistrationYear         0
Gearbox              19833
Power                    0
Model                19705
Kilometer                0
RegistrationMonth        0
FuelType             32895
Brand                    0
Repaired             71154
PostalCode               0
Price                    0
dtype: int64

In [25]:
object_cols = car_prices.select_dtypes(include='object').columns

car_prices[object_cols] = car_prices[object_cols].apply(lambda x: x.fillna(x.mode()[0]))

In [26]:
lower_q, upper_q = car_prices['Price'].quantile([0.08, 0.92])
car_prices = car_prices[(car_prices['Price'] >= lower_q) & (car_prices['Price'] <= upper_q)]
print(f"Shape: {car_prices.shape}")

Shape: (300022, 12)


In [27]:
for col in car_prices.select_dtypes(include='object').columns:
    print(car_prices[col].unique())

['sedan' 'suv' 'small' 'convertible' 'wagon' 'bus' 'coupe' 'other']
['manual' 'auto']
['golf' 'grand' 'fabia' '3er' '2_reihe' '3_reihe' 'passat' 'ka' 'twingo'
 'c_max' 'a_klasse' 'scirocco' '5er' 'meriva' 'arosa' 'other' 'c4' 'civic'
 'transporter' 'punto' 'e_klasse' 'clio' 'kadett' 'kangoo' 'one' 'fortwo'
 '1er' 'signum' 'astra' 'a8' 'jetta' 'polo' 'fiesta' 'c_klasse' 'micra'
 'vito' 'sprinter' 'escort' 'forester' 'xc_reihe' 'scenic' 'a4' 'insignia'
 'combo' 'focus' 'tt' 'corsa' 'a6' 'jazz' 'omega' 'slk' '7er' '80' '147'
 'z_reihe' 'sorento' 'ibiza' 'eos' 'touran' 'getz' 'a3' 'almera' 'megane'
 'lupo' 'r19' 'zafira' 'caddy' 'mondeo' 'cordoba' 'colt' 'impreza' 'a1'
 'vectra' 'berlingo' 'tiguan' 'i_reihe' 'espace' 'sharan' '6_reihe'
 'panda' 'up' 'seicento' 'ceed' 'octavia' 'mii' 'rx_reihe' 'modus' 'fox'
 'matiz' 'beetle' 'c1' 'rio' 'logan' 'spider' 'cuore' 's_max' 'a2' 'c3'
 '1_reihe' 'avensis' 'roomster' 'sl' 'kaefer' 'santa' 'cooper' 'leon'
 '4_reihe' 'sportage' '500' 'laguna' 'ptcru

In [28]:
car_prices['RegistrationYear'] = pd.to_numeric(car_prices['RegistrationYear'], errors='coerce')
car_prices['RegistrationMonth'] = pd.to_numeric(car_prices['RegistrationMonth'], errors='coerce')

In [29]:
current_year = date.today().year

# Filter RegistrationYear
car_prices = car_prices[
    (car_prices['RegistrationYear'] <= current_year) &
    (car_prices['RegistrationYear'] > 1900)
].copy()

# Replace invalid months with median
median_month = int(car_prices.loc[car_prices['RegistrationMonth'] > 0, 'RegistrationMonth'].median())
car_prices['RegistrationMonth'] = car_prices['RegistrationMonth'].apply(lambda x: median_month if x <= 0 else x)

In [30]:
car_prices['RegistrationDate'] = pd.to_datetime(
    car_prices['RegistrationYear'].astype(str) + '-' +
    car_prices['RegistrationMonth'].astype(str),
    format='%Y-%m'
)

car_prices['CarAge'] = ((pd.to_datetime('today') - car_prices['RegistrationDate']).dt.days / 365.25).round(4)

car_prices.drop(columns=['RegistrationYear', 'RegistrationMonth', 'RegistrationDate'], inplace=True, errors='coerce')

In [31]:
num_cols = car_prices.select_dtypes(include=['int64', 'float64']).columns
skew_values = car_prices[num_cols].skew().sort_values(ascending=False)
print(skew_values.round(4))

Power         62.7176
Price          1.0380
CarAge         0.9104
PostalCode     0.0795
Kilometer     -1.9269
dtype: float64


In [32]:
cleaned_car_prices = car_prices.copy()
cleaned_car_prices.to_csv("CleanedCarPrices.csv", index=False)

In [33]:
pt = PowerTransformer(method='yeo-johnson')
num_cols_features=['Power', 'Kilometer', 'CarAge']
car_prices[num_cols_features] = pt.fit_transform(car_prices[num_cols_features])
print(car_prices[num_cols_features].skew().round(4))

Power        1.6723
Kilometer   -1.1625
CarAge       0.1201
dtype: float64


In [34]:
X = car_prices.drop(columns='Price')
Y = car_prices['Price']

In [35]:
low_cardinality_cols = ['Gearbox', 'Repaired', 'FuelType', 'VehicleType']
ohe = OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False)
ohe_df = pd.DataFrame(ohe.fit_transform(X[low_cardinality_cols]), 
                    columns=ohe.get_feature_names_out(low_cardinality_cols),
                    index=X.index)
X = X.drop(columns=low_cardinality_cols)
X = pd.concat([X, ohe_df], axis=1)

In [36]:
high_cardinality_cols = ['Brand', 'Model']
target_encoder = TargetEncoder(cols=high_cardinality_cols)
X[high_cardinality_cols] = target_encoder.fit_transform(X[high_cardinality_cols], Y)

In [37]:
X_train, X_test, Y_train, Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

In [38]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [39]:
# Optimized Gradient Boosting - faster with similar performance
gbr_fast = GradientBoostingRegressor(
    n_estimators=800,  # Reduced but with higher learning rate
    learning_rate=0.1,  # Increased for faster convergence
    max_depth=5,        # Reduced for speed and generalization
    min_samples_split=8,
    min_samples_leaf=5,
    subsample=0.9,
    max_features=0.8,   # Slightly more features
    loss='huber',
    validation_fraction=0.15,
    n_iter_no_change=25,
    tol=1e-4,
    random_state=42
)

# Optimized CatBoost - better trade-off
cat_fast = CatBoostRegressor(
    iterations=1500,           # Reduced iterations
    learning_rate=0.08,        # Slightly adjusted
    depth=7,                   # Reduced depth
    l2_leaf_reg=5,
    subsample=0.85,
    random_strength=0.1,       # Added for regularization
    eval_metric='RMSE',
    random_state=42,
    verbose=0,                 # Less verbose for cleaner output
    early_stopping_rounds=100,
    loss_function='MAE',
    thread_count=-1            # Use all cores
)

# Optimized XGBoost - faster and more accurate
xgb_fast = XGBRegressor(
    n_estimators=1200,
    learning_rate=0.07,
    max_depth=7,              # Reduced depth
    subsample=0.9,
    colsample_bytree=0.85,
    reg_alpha=0.3,
    reg_lambda=0.7,
    gamma=0.1,                # Added for regularization
    random_state=42,
    n_jobs=-1,
    tree_method="hist",       # Faster histogram method
    enable_categorical=False
)

# Optimized LightGBM - fastest and very accurate
lgbm_fast = LGBMRegressor(
    n_estimators=2000,        # More estimators but faster
    learning_rate=0.05,
    max_depth=-1,             # Use -1 for no limit but control with num_leaves
    num_leaves=63,            # Optimal for performance
    subsample=0.9,
    colsample_bytree=0.85,
    reg_alpha=0.2,
    reg_lambda=0.4,
    min_child_samples=30,
    min_split_gain=0.01,      # Added for regularization
    random_state=42,
    n_jobs=-1,
    verbose=-1
)

# Add a diverse model - Random Forest for stability
rf = RandomForestRegressor(
    n_estimators=300,
    max_depth=12,
    min_samples_split=5,
    min_samples_leaf=2,
    max_features=0.7,
    random_state=42,
    n_jobs=-1,
    bootstrap=True
)

meta_improved = ElasticNet(alpha=0.01, l1_ratio=0.7, random_state=42, max_iter=2000)

# Stacking model with improved configuration
stack_model_fast = StackingRegressor(
    estimators=[
        ('lgbm', lgbm_fast),    # Put fastest first
        ('xgb', xgb_fast),
        ('cat', cat_fast),
        ('gbr', gbr_fast),
        ('rf', rf)              # Added for diversity
    ],
    final_estimator=meta_improved,
    passthrough=True,
    cv=3,                       # Reduced for speed, increase to 5 if you have time
    n_jobs=-1
)

# Optional: Simple weighted average ensemble for even faster inference
class WeightedEnsemble:
    def __init__(self, models, weights=None):
        self.models = models
        if weights is None:
            # Default equal weights
            self.weights = [1/len(models)] * len(models)
        else:
            self.weights = weights
    
    def fit(self, X, Y):
        for model in self.models:
            model.fit(X, Y)
        return self
    
    def predict(self, X):
        predictions = np.zeros(X.shape[0])
        for model, weight in zip(self.models, self.weights):
            predictions += weight * model.predict(X)
        return predictions

# Fast ensemble (use this if stacking is too slow)
fast_ensemble = WeightedEnsemble(
    models=[lgbm_fast, xgb_fast, cat_fast],
    weights=[0.4, 0.35, 0.25]  # Tune based on cross-validation: [lgbm, xgb, rf]
)

fast_ensemble.fit(X_train, Y_train)
Y_pred_stack = fast_ensemble.predict(X_test)

r2_stack = r2_score(Y_test, Y_pred_stack)
r2_mae = mean_absolute_error(Y_test, Y_pred_stack)
r2_mse = mean_squared_error(Y_test, Y_pred_stack)
r2_rmse = root_mean_squared_error(Y_test, Y_pred_stack)

print(f"Stacking Regressor R2 Score: {round(r2_stack, 2)}")
print(f"Stacking Regressor MAE: {round(r2_mae, 2)}")
print(f"Stacking Regressor MSE: {round(r2_mse, 2)}")
print(f"Stacking Regressor RMSE: {round(r2_rmse, 2)}")

c:\Users\Abhinav\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


Stacking Regressor R2 Score: 0.85
Stacking Regressor MAE: 809.31
Stacking Regressor MSE: 1473164.96
Stacking Regressor RMSE: 1213.74


In [40]:
with open('CarPricePrediction.pkl', 'wb') as f:
    pkl.dump(fast_ensemble, f)

In [41]:
def interactive_quick_predict():
    """Interactive version of quick_predict - asks for inputs one by one"""
    
    print("🚗 Car Price Prediction - Enter Car Details")
    print("=" * 40)
    
    # Collect all inputs interactively
    input_data = {
        'VehicleType': input("Enter Vehicle Type (limousine, suv, small_car, etc.): "),
        'Gearbox': input("Enter Gearbox (manual, automatic): "),
        'FuelType': input("Enter Fuel Type (petrol, diesel, electric, etc.): "),
        'Power': int(input("Enter Power (HP): ")),
        'Kilometer': int(input("Enter Kilometer: ")),
        'Brand': input("Enter Brand (volkswagen, bmw, mercedes_benz, etc.): "),
        'Model': input("Enter Model (golf, x5, c_class, etc.): "),
        'Repaired': input("Enter Repaired (yes, no): "),
        'PostalCode': int(input("Enter Postal Code: ")),
        'RegistrationDate': input("Enter Registration Date (YYYY-MM, e.g., 2015-06): ")
    }
    
    print("\n⏳ Processing your car data...")
    
    # Convert to DataFrame
    input_df = pd.DataFrame([input_data])
    
    # Calculate CarAge if needed
    if 'RegistrationDate' in input_df.columns and 'CarAge' not in input_df.columns:
        input_df['CarAge'] = input_df['RegistrationDate'].apply(
            lambda x: (pd.to_datetime('today') - pd.to_datetime(x, format='%Y-%m')).days / 365.25
        )
    
    # Fill missing columns
    for col in num_cols_features:
        if col not in input_df.columns:
            input_df[col] = 0
    
    # Apply transformations
    input_df[num_cols_features] = pt.transform(input_df[num_cols_features])
    
    # One-hot encoding
    for col in low_cardinality_cols:
        if col not in input_df.columns:
            input_df[col] = 'unknown'
    
    input_ohe = pd.DataFrame(
        ohe.transform(input_df[low_cardinality_cols]),
        columns=ohe.get_feature_names_out(low_cardinality_cols),
        index=input_df.index
    )
    input_df = input_df.drop(columns=low_cardinality_cols)
    input_df = pd.concat([input_df, input_ohe], axis=1)
    
    # Target encoding
    for col in high_cardinality_cols:
        if col not in input_df.columns:
            input_df[col] = 'unknown'
    
    input_df[high_cardinality_cols] = target_encoder.transform(input_df[high_cardinality_cols])
    
    # Align columns and predict
    input_df = input_df.reindex(columns=X.columns, fill_value=0)
    input_scaled = scaler.transform(input_df)
    
    predicted_price_eur = fast_ensemble.predict(input_scaled)[0]
    predicted_price_inr = predicted_price_eur * 102.88
    
    # Display results
    print("\n" + "🎯" * 20)
    print("      PREDICTION RESULTS")
    print("🎯" * 20)
    print(f"Car: {input_data['Brand'].title()} {input_data['Model'].title()}")
    print(f"Year: {input_data['RegistrationDate']}")
    print(f"Mileage: {input_data['Kilometer']:,} km")
    print(f"Power: {input_data['Power']} HP")
    print(f"Fuel: {input_data['FuelType'].title()}")
    print(f"Transmission: {input_data['Gearbox'].title()}")
    print("-" * 40)
    print(f"💰 Predicted Price: €{predicted_price_eur:,.2f}")
    print(f"💰 Predicted Price: ₹{predicted_price_inr:,.2f}")
    print("=" * 40)
    
    return predicted_price_eur, predicted_price_inr

# Run the interactive predictor
eur_price, inr_price = interactive_quick_predict()

# Keep your original function as is
def quick_predict(input_data):
    """Quick prediction without detailed steps"""
    
    # Convert to DataFrame
    input_df = pd.DataFrame([input_data])
    
    # Calculate CarAge if needed
    if 'RegistrationDate' in input_df.columns and 'CarAge' not in input_df.columns:
        input_df['CarAge'] = input_df['RegistrationDate'].apply(
            lambda x: (pd.to_datetime('today') - pd.to_datetime(x, format='%Y-%m')).days / 365.25
        )
    
    # Fill missing columns
    for col in num_cols_features:
        if col not in input_df.columns:
            input_df[col] = 0
    
    # Apply transformations
    input_df[num_cols_features] = pt.transform(input_df[num_cols_features])
    
    # One-hot encoding
    for col in low_cardinality_cols:
        if col not in input_df.columns:
            input_df[col] = 'unknown'
    
    input_ohe = pd.DataFrame(
        ohe.transform(input_df[low_cardinality_cols]),
        columns=ohe.get_feature_names_out(low_cardinality_cols),
        index=input_df.index
    )
    input_df = input_df.drop(columns=low_cardinality_cols)
    input_df = pd.concat([input_df, input_ohe], axis=1)
    
    # Target encoding
    for col in high_cardinality_cols:
        if col not in input_df.columns:
            input_df[col] = 'unknown'
    
    input_df[high_cardinality_cols] = target_encoder.transform(input_df[high_cardinality_cols])
    
    # Align columns and predict
    input_df = input_df.reindex(columns=X.columns, fill_value=0)
    input_scaled = scaler.transform(input_df)
    
    predicted_price_eur = fast_ensemble.predict(input_scaled)[0]
    predicted_price_inr = predicted_price_eur * 102.88
    
    return predicted_price_eur, predicted_price_inr

🚗 Car Price Prediction - Enter Car Details

⏳ Processing your car data...

🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯
      PREDICTION RESULTS
🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯
Car: Bmw C_Class
Year: 2022-03
Mileage: 150,000 km
Power: 500 HP
Fuel: Petrol
Transmission: Manual
----------------------------------------
💰 Predicted Price: €5,228.10
💰 Predicted Price: ₹537,867.30


c:\Users\Abhinav\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
